In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from calculations.calculations import get_company_name
from datetime import timedelta

In [11]:
stock = yf.download('^SPX', period='3mo', interval='1h')
data = stock.tz_convert('CET')

[*********************100%%**********************]  1 of 1 completed


In [12]:
# Convertir índice a Datetime
data['Datetime'] = pd.to_datetime(data.index)

# Crear subgráfico
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Agregar vela japonesa
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'],
                             name='Candlestick'))

# Determinar el intervalo de datos
interval = '1h'
print(f"Intervalo detectado: {interval}")

# Identificar el primer día laborable
first_day = data['Datetime'].iloc[0]
first_day_working_hours = data[(data['Datetime'].dt.date == first_day.date())]

# Obtener las horas activas del primer día laborable
active_hours = first_day_working_hours['Datetime'].dt.time.unique()

# Crear lista de fechas y horas activas
trading_hours = [pd.Timestamp.combine(day, time) for day in pd.date_range(start=data['Datetime'].min().date(), 
                                                                         end=data['Datetime'].max().date(), 
                                                                         freq='B') 
                 for time in active_hours]

# Verificar si se tienen datos en fines de semana
has_weekend_data = any(data['Datetime'].dt.weekday >= 5)

# Crear el conjunto de tiempos activos solo si no hay datos en fines de semana
if not has_weekend_data:
    # Crear un conjunto de todas las fechas laborables y horas activas
    trading_hours = [pd.Timestamp.combine(day, time) for day in pd.date_range(start=data['Datetime'].min().date(), 
                                                                             end=data['Datetime'].max().date(), 
                                                                             freq='B') 
                     for time in active_hours]

# Identificar los tiempos faltantes
missing_times = set(trading_hours) - set(data['Datetime'])

Intervalo detectado: 1h


In [14]:
trading_hours

[Timestamp('2024-04-22 15:30:00'),
 Timestamp('2024-04-22 16:30:00'),
 Timestamp('2024-04-22 17:30:00'),
 Timestamp('2024-04-22 18:30:00'),
 Timestamp('2024-04-22 19:30:00'),
 Timestamp('2024-04-22 20:30:00'),
 Timestamp('2024-04-22 21:30:00'),
 Timestamp('2024-04-23 15:30:00'),
 Timestamp('2024-04-23 16:30:00'),
 Timestamp('2024-04-23 17:30:00'),
 Timestamp('2024-04-23 18:30:00'),
 Timestamp('2024-04-23 19:30:00'),
 Timestamp('2024-04-23 20:30:00'),
 Timestamp('2024-04-23 21:30:00'),
 Timestamp('2024-04-24 15:30:00'),
 Timestamp('2024-04-24 16:30:00'),
 Timestamp('2024-04-24 17:30:00'),
 Timestamp('2024-04-24 18:30:00'),
 Timestamp('2024-04-24 19:30:00'),
 Timestamp('2024-04-24 20:30:00'),
 Timestamp('2024-04-24 21:30:00'),
 Timestamp('2024-04-25 15:30:00'),
 Timestamp('2024-04-25 16:30:00'),
 Timestamp('2024-04-25 17:30:00'),
 Timestamp('2024-04-25 18:30:00'),
 Timestamp('2024-04-25 19:30:00'),
 Timestamp('2024-04-25 20:30:00'),
 Timestamp('2024-04-25 21:30:00'),
 Timestamp('2024-04-

In [ ]:
# Actualizar rangos de ejes para excluir tiempos faltantes
fig.update_xaxes(rangebreaks=[dict(values=list(missing_times))])

# Actualizar el layout del gráfico
fig.update_layout(title=f'SPX', 
                  xaxis_title='', yaxis_title='',
                  yaxis=dict(gridcolor='rgba(200, 200, 200, 0.03)', gridwidth=1),
                  xaxis_rangeslider_visible=False,
                  height=700,
                  dragmode='drawline',
                  shapes=[],
                  newshape=dict(line=dict(color="red")),
                  modebar_add=['drawline','eraseshape'],
                  showlegend=False)

fig.show()